In [7]:
#爬取数据获取榜单创建json
from datetime import datetime
import json
import os
import re

from lxml import etree
import requests

import utils


BASE_URL = 'https://s.weibo.com'
JSON_DIR = './raw'
ARCHIVE_DIR = './archives'


def getHTML(url):
    ''' 获取网页 HTML 返回字符串

    Args:
        url: str, 网页网址
    Returns:
        HTML 字符串
    '''
    # Cookie 有效期至2023-02-10
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36',
        'Cookie': 'SUB=_2AkMVWDYUf8NxqwJRmP0Sz2_hZYt2zw_EieKjBMfPJRMxHRl-yj9jqkBStRB6PtgY-38i0AF7nDAv8HdY1ZwT3Rv8B5e5; SUBP=0033WrSXqPxfM72-Ws9jqgMF55529P9D9WFencmWZyNhNlrzI6f0SiqP'
    }
    response = requests.get(url, headers=headers)
    if response.encoding == 'ISO-8859-1':
        response.encoding = response.apparent_encoding if response.apparent_encoding != 'ISO-8859-1' else 'utf-8'
    return response.text


# 使用 xpath 解析 HTML
def parseHTMLByXPath(content):
    ''' 使用 xpath 解析 HTML, 提取榜单信息

    Args:
        content: str, 待解析的 HTML 字符串
    Returns:
        榜单信息的字典 字典
    '''
    html = etree.HTML(content)

    titles = html.xpath(
        '//tr[position()>1]/td[@class="td-02"]/a[not(contains(@href, "javascript:void(0);"))]/text()')
    hrefs = html.xpath(
        '//tr[position()>1]/td[@class="td-02"]/a[not(contains(@href, "javascript:void(0);"))]/@href')
    hots = html.xpath(
        '//tr[position()>1]/td[@class="td-02"]/a[not(contains(@href, "javascript:void(0);"))]/../span/text()')
    titles = [title.strip() for title in titles]
    hrefs = [BASE_URL + href.strip() for href in hrefs]
    hots = [int(hot.strip().split(' ')[-1])
            for hot in hots]  # 该处除了热度还会返回大致分类，形如 `剧集 53412536`，前为分类，后为热度

    correntRank = {}
    for i, title in enumerate(titles):
        correntRank[title] = {'href': hrefs[i], 'hot': hots[i]}

    return correntRank


# 更新本日榜单
def updateJSON(correntRank):
    ''' 更新当天的 JSON 文件

    Args:
        correntRank: dict, 最新的榜单信息
    Returns:
        与当天历史榜单对比去重, 排序后的榜单信息字典
    '''
    filename = datetime.today().strftime('%Y%m%d') + '.json'
    filename = os.path.join(JSON_DIR, filename)

    # 文件不存在则创建
    if not os.path.exists(filename):
        utils.save(filename, {})

    historyRank = json.loads(utils.load(filename))
    for k, v in correntRank.items():
        # 若当前榜单和历史榜单有重复的，取热度数值(名称后面的数值)更大的一个
        if k in historyRank:
            historyRank[k]['hot'] = max(
                historyRank[k]['hot'], correntRank[k]['hot'])
        # 若没有，则添加
        else:
            historyRank[k] = v

    # 将榜单按 hot 值排序
    rank = {k: v for k, v in sorted(
        historyRank.items(), key=lambda item: item[1]['hot'], reverse=True)}

    # 更新当天榜单 json 文件
    utils.save(filename, rank)
    return rank


def updateReadme(rank):
    ''' 更新 README.md

    Args:
        rank: dict, 榜单信息
    Returns:
        None
    '''
    filename = './123.txt'

    line = '1. [{title}]({href}) {hot}'
    lines = [line.format(title=k, hot=v['hot'], href=v['href'])
             for k, v in rank.items()]
    rank = '\n'.join(lines)

    rank = '最后更新时间 {}\n\n'.format(
        datetime.now().strftime('%Y-%m-%d %X')) + rank
    rank = '<!-- Rank Begin -->\n\n' + rank + '\n<!-- Rank End -->'

    content = re.sub(
        r'<!-- Rank Begin -->[\s\S]*<!-- Rank End -->', rank, utils.load(filename))
    utils.save(filename, content)
    print(rank,content)

def main():
    url = '/top/summary'

    content = getHTML(BASE_URL + url)
    # with open('test.html', 'w', encoding='utf-8') as f:
    #     f.write(content)
    # print(content)
    correntRank = parseHTMLByXPath(content)
    rankJSON = updateJSON(correntRank)
    updateReadme(rankJSON)


if __name__ == '__main__':
    main()


['2022-02-11',
 '2022-02-12',
 '2022-02-13',
 '2022-02-14',
 '2022-02-15',
 '2022-02-16',
 '2022-02-17',
 '2022-02-18',
 '2022-02-19',
 '2022-02-20',
 '2022-02-21',
 '2022-02-22',
 '2022-02-23',
 '2022-02-24',
 '2022-02-25',
 '2022-02-26',
 '2022-02-27',
 '2022-02-28',
 '2022-03-01',
 '2022-03-02',
 '2022-03-03',
 '2022-03-04',
 '2022-03-05',
 '2022-03-06',
 '2022-03-07',
 '2022-03-08',
 '2022-03-09',
 '2022-03-10',
 '2022-03-11',
 '2022-03-12',
 '2022-03-13',
 '2022-03-14',
 '2022-03-15',
 '2022-03-16',
 '2022-03-17',
 '2022-03-18',
 '2022-03-19',
 '2022-03-20',
 '2022-03-21',
 '2022-03-22',
 '2022-03-23',
 '2022-03-24',
 '2022-03-25',
 '2022-03-26',
 '2022-03-27',
 '2022-03-28',
 '2022-03-29',
 '2022-03-30',
 '2022-03-31',
 '2022-04-01',
 '2022-04-02',
 '2022-04-03',
 '2022-04-04',
 '2022-04-05',
 '2022-04-06',
 '2022-04-07',
 '2022-04-08',
 '2022-04-09',
 '2022-04-10',
 '2022-04-11',
 '2022-04-12',
 '2022-04-13',
 '2022-04-14',
 '2022-04-15',
 '2022-04-16',
 '2022-04-17',
 '2022-04-

In [50]:
import os
import re
path_list=[]
date_list=[]
file = "raw/"

for root, dirs, files in os.walk(file):
    for file in files:
        path = os.path.join(root, file)
        print(path)
        path_list.append(path)
        m = re.findall("\d+", path)
        a=m[0][0:4]+"-"+m[0][4:6]+"-"+m[0][6:8]
        date_list.append(a)


raw/20220301.json
raw/20220614.json
raw/20221005.json
raw/20220706.json
raw/20220213.json
raw/20221228.json
raw/20221101.json
raw/20220710.json
raw/20220317.json
raw/20221013.json
raw/20220602.json
raw/20220415.json
raw/20220803.json
raw/20220911.json
raw/20220507.json
raw/20220907.json
raw/20220511.json
raw/20221029.json
raw/20220403.json
raw/20220815.json
raw/20221009.json
raw/20220618.json
raw/20220531.json
raw/20220927.json
raw/20220423.json
raw/20220823.json
raw/20221224.json
raw/20220527.json
raw/20220225.json
raw/20220730.json
raw/20220419.json
raw/20220622.json
raw/20221025.json
raw/20220321.json
raw/20220726.json
raw/20220819.json
raw/20220818.json
raw/20220727.json
raw/20221024.json
raw/20220320.json
raw/20220623.json
raw/20220418.json
raw/20220224.json
raw/20220731.json
raw/20220526.json
raw/20220930.json
raw/20220822.json
raw/20221225.json
raw/20220422.json
raw/20220530.json
raw/20220926.json
raw/20221008.json
raw/20220619.json
raw/20220402.json
raw/20220814.json
raw/202210

In [1]:
import pymysql
# 引入python中的traceback模块，跟踪错误
import traceback
import sys


class MysqlUtil():
    def __init__(self):
        '''
            初始化方法，连接数据库
        '''
        host = '192.168.8.236'  # 主机名
        user = 'root'  # 数据库用户名
        password = '8008'  # 数据库密码
        database = 'weibo'  # 数据库名称
        self.db = pymysql.connect(host=host, user=user, password=password, db=database)  # 建立连接
        self.cursor = self.db.cursor(cursor=pymysql.cursors.DictCursor)  # 设置游标，并将游标设置为字典类型

    def insert(self, sql):
        '''
            插入数据库
            sql:插入数据库的sql语句
        '''
        try:
            # 执行sql语句
            self.cursor.execute(sql)
            # 提交到数据库执行
            self.db.commit()
        except Exception:  # 方法一：捕获所有异常
            # 如果发生异常，则回滚
            print("发生异常", Exception)
            self.db.rollback()
        finally:
            # 最终关闭数据库连接
            self.db.close()

    def fetchone(self, sql):
        '''
            查询数据库：单个结果集
            fetchone(): 该方法获取下一个查询结果集。结果集是一个对象
        '''
        try:
            # 执行sql语句
            self.cursor.execute(sql)
            result = self.cursor.fetchone()
        except:  # 方法二：采用traceback模块查看异常
            # 输出异常信息
            traceback.print_exc()
            # 如果发生异常，则回滚
            self.db.rollback()
            result = 'None'

        finally:
            # 最终关闭数据库连接
            self.db.close()
        return result

    def fetchall(self, sql):
        '''
            查询数据库：多个结果集
            fetchall(): 接收全部的返回结果行.
        '''
        try:
            # 执行sql语句
            self.cursor.execute(sql)
            results = self.cursor.fetchall()
        except:  # 方法三：采用sys模块回溯最后的异常
            # 输出异常信息
            info = sys.exc_info()
            print(info[0], ":", info[1])
            # 如果发生异常，则回滚
            self.db.rollback()
            results = 'None'
        finally:
            # 最终关闭数据库连接
            self.db.close()
        return results

    def delete(self, sql):
        '''
            删除结果集
        '''
        try:
            # 执行sql语句
            self.cursor.execute(sql)
            self.db.commit()
        except:  # 把这些异常保存到一个日志文件中，来分析这些异常
            # 将错误日志输入到目录文件中
            f = open("\log.txt", 'a')
            traceback.print_exc(file=f)
            f.flush()
            f.close()
            # 如果发生异常，则回滚
            self.db.rollback()
        finally:
            # 最终关闭数据库连接
            self.db.close()

    def update(self, sql):
        '''
            更新结果集
        '''
        try:
            # 执行sql语句
            self.cursor.execute(sql)
            self.db.commit()
        except:
            # 如果发生异常，则回滚
            self.db.rollback()
        finally:
            # 最终关闭数据库连接
            self.db.close()


In [51]:
import json
import time
import pandas as pd
for i in range(len(path_list)):
    print("第"+str(i)+"天")
    with open(path_list[i], encoding='utf-8') as f:
        json_data = json.load(f)
    sss=json_data
    lst_key=[]
    lst_value=[]
    for key,value in sss.items():
        lst_key.append(key)
        lst_value.append(value)
    for kk in range(len(lst_key)):
        print("第"+str(kk)+"个")
        create_date = time.strftime("%Y-%m-%d", time.localtime())
        sql = "INSERT INTO weibohotpot(title,url,hot,create_date) \
                          VALUES ('%s', '%s', %d,'%s')" % (lst_key[kk], lst_value[kk]['href'], lst_value[kk]['hot'], date_list[i])
        db = MysqlUtil()
        db.insert(sql)



第0天
第0个
第1个
第2个
第3个
第4个
第5个
第6个
第7个
第8个
第9个
第10个
第11个
第12个
第13个
第14个
第15个
第16个
第17个
第18个
第19个
第20个
第21个
第22个
第23个
第24个
第25个
第26个
第27个
第28个
第29个
第30个
第31个
第32个
第33个
第34个
第35个
第36个
第37个
第38个
第39个
第40个
第41个
第42个
第43个
第44个
第45个
第46个
第47个
第48个
第49个
第50个
第51个
第52个
第53个
第54个
第55个
第56个
第57个
第58个
第59个
第60个
第61个
第62个
第63个
第64个
第65个
第66个
第67个
第68个
第69个
第70个
第71个
第72个
第73个
第74个
第75个
第76个
第77个
第78个
第79个
第80个
第81个
第82个
第83个
第84个
第85个
第86个
第87个
第88个
第89个
第90个
第91个
第92个
第93个
第94个
第95个
第96个
第97个
第98个
第99个
第100个
第101个
第102个
第103个
第104个
第105个
第106个
第107个
第108个
第109个
第110个
第111个
第112个
第113个
第114个
第115个
第116个
第117个
第118个
第119个
第120个
第121个
第122个
第123个
第124个
第125个
第126个
第127个
第128个
第129个
第130个
第131个
第132个
第133个
第134个
第135个
第136个
第137个
第138个
第139个
第140个
第141个
第142个
第143个
第144个
第145个
第146个
第147个
第148个
第149个
第150个
第151个
第152个
第153个
第154个
第155个
第156个
第157个
第158个
第159个
第160个
第161个
第162个
第163个
第164个
第165个
第166个
第167个
第168个
第169个
第170个
第171个
第172个
第173个
第174个
第175个
第176个
第177个
第178个
第179个
第180个
第181个
第182个
第183个
第1

第1个
第2个
第3个
第4个
第5个
第6个
第7个
第8个
第9个
第10个
第11个
第12个
第13个
第14个
第15个
第16个
第17个
第18个
第19个
第20个
第21个
第22个
第23个
第24个
第25个
第26个
第27个
第28个
第29个
第30个
第31个
第32个
第33个
第34个
第35个
第36个
第37个
第38个
第39个
第40个
第41个
第42个
第43个
第44个
第45个
第46个
第47个
第48个
第49个
第50个
第51个
第52个
第53个
第54个
第55个
第56个
第57个
第58个
第59个
第60个
第61个
第62个
第63个
第64个
第65个
第66个
第67个
第68个
第69个
第70个
第71个
第72个
第73个
第74个
第75个
第76个
第77个
第78个
第79个
第80个
第81个
第82个
第83个
第84个
第85个
第86个
第87个
第88个
第89个
第90个
第91个
第92个
第93个
第94个
第95个
第96个
第97个
第98个
第99个
第100个
第101个
第102个
第103个
第104个
第105个
第106个
第107个
第108个
第109个
第110个
第111个
第112个
第113个
第114个
第115个
第116个
第117个
第118个
第119个
第120个
第121个
第122个
第123个
第124个
第125个
第126个
第127个
第128个
第129个
第130个
第131个
第132个
第133个
第134个
第135个
第136个
第137个
第138个
第139个
第140个
第141个
第142个
第143个
第144个
第145个
第146个
第147个
第148个
第149个
第150个
第151个
第152个
第153个
第154个
第155个
第156个
第157个
第158个
第159个
第160个
第161个
第162个
第163个
第164个
第165个
第166个
第167个
第168个
第169个
第170个
第171个
第172个
第173个
第174个
第175个
第176个
第177个
第178个
第179个
第180个
第181个
第182个
第183个
第184个
第185

第75个
第76个
第77个
第78个
第79个
第80个
第81个
第82个
第83个
第84个
第85个
第86个
第87个
第88个
第89个
第90个
第91个
第92个
第93个
第94个
第95个
第96个
第97个
第98个
第99个
第100个
第101个
第102个
第103个
第104个
第105个
第106个
第107个
第108个
第109个
第110个
第111个
第112个
第113个
第114个
第115个
第116个
第117个
第118个
第119个
第120个
第121个
第122个
第123个
第124个
第125个
第126个
第127个
第128个
第129个
第130个
第131个
第132个
第133个
第134个
第135个
第136个
第137个
第138个
第139个
第140个
第141个
第142个
第143个
第144个
第145个
第146个
第147个
第148个
第149个
第150个
第151个
第152个
第153个
第154个
第155个
第156个
第157个
第158个
第159个
第160个
第161个
第162个
第163个
第164个
第165个
第166个
第167个
第168个
第169个
第170个
第171个
第172个
第173个
第174个
第175个
第176个
第177个
第178个
第179个
第180个
第181个
第182个
第183个
第184个
第185个
第186个
第187个
第188个
第189个
第190个
第191个
第192个
第193个
第194个
第195个
第196个
第197个
第198个
第199个
第200个
第201个
第202个
第203个
第204个
第205个
第206个
第207个
第208个
第209个
第210个
第211个
第212个
第213个
第214个
第215个
第216个
第217个
第218个
第219个
第220个
第221个
第222个
第223个
第224个
第225个
第226个
第227个
第228个
第229个
第230个
第231个
第232个
第233个
第234个
第235个
第236个
第237个
第238个
第239个
第240个
第241个
第242个
第243个
第244个
第245个

第89个
第90个
第91个
第92个
第93个
第94个
第95个
第96个
第97个
第98个
第99个
第100个
第101个
第102个
第103个
第104个
第105个
第106个
第107个
第108个
第109个
第110个
第111个
第112个
第113个
第114个
第115个
第116个
第117个
第118个
第119个
第120个
第121个
第122个
第123个
第124个
第125个
第126个
第127个
第128个
第129个
第130个
第131个
第132个
第133个
第134个
第135个
第136个
第137个
第138个
第139个
第140个
第141个
第142个
第143个
第144个
第145个
第146个
第147个
第148个
第149个
第150个
第151个
第152个
第153个
第154个
第155个
第156个
第157个
第158个
第159个
第160个
第161个
第162个
第163个
第164个
第165个
第166个
第167个
第168个
第169个
第170个
第171个
第172个
第173个
第174个
第175个
第176个
第177个
第178个
第179个
第180个
第181个
第182个
第183个
第184个
第185个
第186个
第187个
第188个
第189个
第190个
第191个
第192个
第193个
第194个
第195个
第196个
第197个
第198个
第199个
第200个
第201个
第202个
第203个
第204个
第205个
第206个
第207个
第208个
第209个
第210个
第211个
第212个
第213个
第214个
第215个
第216个
第217个
第218个
第219个
第220个
第221个
第222个
第223个
第224个
第225个
第226个
第227个
第228个
第229个
第230个
第231个
第232个
第233个
第234个
第235个
第236个
第237个
第238个
第239个
第240个
第241个
第242个
第243个
第244个
第245个
第246个
第247个
第248个
第249个
第250个
第251个
第252个
第253个
第254个
第255个
第256个
第25

第2个
第3个
第4个
第5个
第6个
第7个
第8个
第9个
第10个
第11个
第12个
第13个
第14个
第15个
第16个
第17个
第18个
第19个
第20个
第21个
第22个
第23个
第24个
第25个
第26个
第27个
第28个
第29个
第30个
第31个
第32个
第33个
第34个
第35个
第36个
第37个
第38个
第39个
第40个
第41个
第42个
第43个
第44个
第45个
第46个
第47个
第48个
第49个
第50个
第51个
第52个
第53个
第54个
第55个
第56个
第57个
第58个
第59个
第60个
第61个
第62个
第63个
第64个
第65个
第66个
第67个
第68个
第69个
第70个
第71个
第72个
第73个
第74个
第75个
第76个
第77个
第78个
第79个
第80个
第81个
第82个
第83个
第84个
第85个
第86个
第87个
第88个
第89个
第90个
第91个
第92个
第93个
第94个
第95个
第96个
第97个
第98个
第99个
第100个
第101个
第102个
第103个
第104个
第105个
第106个
第107个
第108个
第109个
第110个
第111个
第112个
第113个
第114个
第115个
第116个
第117个
第118个
第119个
第120个
第121个
第122个
第123个
第124个
第125个
第126个
第127个
第128个
第129个
第130个
第131个
第132个
第133个
第134个
第135个
第136个
第137个
第138个
第139个
第140个
第141个
第142个
第143个
第144个
第145个
第146个
第147个
第148个
第149个
第150个
第151个
第152个
第153个
第154个
第155个
第156个
第157个
第158个
第159个
第160个
第161个
第162个
第163个
第164个
第165个
第166个
第167个
第168个
第169个
第170个
第171个
第172个
第173个
第174个
第175个
第176个
第177个
第178个
第179个
第180个
第181个
第182个
第183个
第184个
第185个
第1

...

第80个
第81个
第82个
第83个
第84个
第85个
第86个
第87个
第88个
第89个
第90个
第91个
第92个
第93个
第94个
第95个
第96个
第97个
第98个
第99个
第100个
第101个
第102个
第103个
第104个
第105个
第106个
第107个
第108个
第109个
第110个
第111个
第112个
第113个
第114个
第115个
第116个
第117个
第118个
第119个
第120个
第121个
第122个
第123个
第124个
第125个
第126个
第127个
第128个
第129个
第130个
第131个
第132个
第133个
第134个
第135个
第136个
第137个
第138个
第139个
第140个
第141个
第142个
第143个
第144个
第145个
第146个
第147个
第148个
第149个
第150个
第151个
第152个
第153个
第154个
第155个
第156个
第157个
第158个
第159个
第160个
第161个
第162个
第163个
第164个
第165个
第166个
第167个
第168个
第169个
第170个
第171个
第172个
第173个
第174个
第175个
第176个
第177个
第178个
第179个
第180个
第181个
第182个
第183个
第184个
第185个
第186个
第187个
第188个
第189个
第190个
第191个
第192个
第193个
第194个
第195个
第196个
第197个
第198个
第199个
第200个
第201个
第202个
第203个
第204个
第205个
第206个
第207个
第208个
第209个
第210个
第211个
第212个
第213个
第214个
第215个
第216个
第217个
第218个
第219个
第220个
第221个
第222个
第223个
第224个
第225个
第226个
第227个
第228个
第229个
第230个
第231个
第232个
第233个
第234个
第235个
第236个
第237个
第238个
第239个
第240个
第241个
第242个
第243个
第244个
第245个
第246个
第247个
第248个
第249个


第72个
第73个
第74个
第75个
第76个
第77个
第78个
第79个
第80个
第81个
第82个
第83个
第84个
第85个
第86个
第87个
第88个
第89个
第90个
第91个
第92个
第93个
第94个
第95个
第96个
第97个
第98个
第99个
第100个
第101个
第102个
第103个
第104个
第105个
第106个
第107个
第108个
第109个
第110个
第111个
第112个
第113个
第114个
第115个
第116个
第117个
第118个
第119个
第120个
第121个
第122个
第123个
第124个
第125个
第126个
第127个
第128个
第129个
第130个
第131个
第132个
第133个
第134个
第135个
第136个
第137个
第138个
第139个
第140个
第141个
第142个
第143个
第144个
第145个
第146个
第147个
第148个
第149个
第150个
第151个
第152个
第153个
第154个
第155个
第156个
第157个
第158个
第159个
第160个
第161个
第162个
第163个
第164个
第165个
第166个
第167个
第168个
第169个
第170个
第171个
第172个
第173个
第174个
第175个
第176个
第177个
第178个
第179个
第180个
第181个
第182个
第183个
第184个
第185个
第186个
第187个
第188个
第189个
第190个
第191个
第192个
第193个
第194个
第195个
第196个
第197个
第198个
第199个
第200个
第201个
第202个
第203个
第204个
第205个
第206个
第207个
第208个
第209个
第210个
第211个
第212个
第213个
第214个
第215个
第216个
第217个
第218个
第219个
第220个
第221个
第222个
第223个
第224个
第225个
第226个
第227个
第228个
第229个
第230个
第231个
第232个
第233个
第234个
第235个
第236个
第237个
第238个
第239个
第240个
第241个
第242个
第2

第82个
第83个
第84个
第85个
第86个
第87个
第88个
第89个
第90个
第91个
第92个
第93个
第94个
第95个
第96个
第97个
第98个
第99个
第100个
第101个
第102个
第103个
第104个
第105个
第106个
第107个
第108个
第109个
第110个
第111个
第112个
第113个
第114个
第115个
第116个
第117个
第118个
第119个
第120个
第121个
第122个
第123个
第124个
第125个
第126个
第127个
第128个
第129个
第130个
第131个
第132个
第133个
第134个
第135个
第136个
第137个
第138个
第139个
第140个
第141个
第142个
第143个
第144个
第145个
第146个
第147个
第148个
第149个
第150个
第151个
第152个
第153个
第154个
第155个
第156个
第157个
第158个
第159个
第160个
第161个
第162个
第163个
第164个
第165个
第166个
第167个
第168个
第169个
第170个
第171个
第172个
第173个
第174个
第175个
第176个
第177个
第178个
第179个
第180个
第181个
第182个
第183个
第184个
第185个
第186个
第187个
第188个
第189个
第190个
第191个
第192个
第193个
第194个
第195个
第196个
第197个
第198个
第199个
第200个
第201个
第202个
第203个
第204个
第205个
第206个
第207个
第208个
第209个
第210个
第211个
第212个
第213个
第214个
第215个
第216个
第217个
第218个
第219个
第220个
第221个
第222个
第223个
第224个
第225个
第226个
第227个
第228个
第229个
第230个
第231个
第232个
第233个
第234个
第235个
第236个
第237个
第238个
第239个
第240个
第241个
第242个
第243个
第244个
第245个
第246个
第247个
第248个
第249个
第250个
第251

第319个
第320个
第321个
第322个
第323个
第324个
第325个
第326个
第327个
第328个
第329个
第330个
第331个
第332个
第333个
第283天
第0个
第1个
第2个
第3个
第4个
第5个
第6个
第7个
第8个
第9个
第10个
第11个
第12个
第13个
第14个
第15个
第16个
第17个
第18个
第19个
第20个
第21个
第22个
第23个
第24个
第25个
第26个
第27个
第28个
第29个
第30个
第31个
第32个
第33个
第34个
第35个
第36个
第37个
第38个
第39个
第40个
第41个
第42个
第43个
第44个
第45个
第46个
第47个
第48个
第49个
第50个
第51个
第52个
第53个
第54个
第55个
第56个
第57个
第58个
第59个
第60个
第61个
第62个
第63个
第64个
第65个
第66个
第67个
第68个
第69个
第70个
第71个
第72个
第73个
第74个
第75个
第76个
第77个
第78个
第79个
第80个
第81个
第82个
第83个
第84个
第85个
第86个
第87个
第88个
第89个
第90个
第91个
第92个
第93个
第94个
第95个
第96个
第97个
第98个
第99个
第100个
第101个
第102个
第103个
第104个
第105个
第106个
第107个
第108个
第109个
第110个
第111个
第112个
第113个
第114个
第115个
第116个
第117个
第118个
第119个
第120个
第121个
第122个
第123个
第124个
第125个
第126个
第127个
第128个
第129个
第130个
第131个
第132个
第133个
第134个
第135个
第136个
第137个
第138个
第139个
第140个
第141个
第142个
第143个
第144个
第145个
第146个
第147个
第148个
第149个
第150个
第151个
第152个
第153个
第154个
第155个
第156个
第157个
第158个
第159个
第160个
第161个
第162个
第163个
第164个
第165个
第166个
第167个
第168个


第300个
第301个
第302个
第303个
第304个
第305个
第306个
第307个
第308个
第309个
第310个
第311个
第312个
第313个
第314个
第315个
第316个
第317个
第318个
第319个
第320个
第321个
第322个
第323个
第324个
第325个
第326个
第327个
第328个
第329个
第330个
第331个
第332个
第288天
第0个
第1个
第2个
第3个
第4个
第5个
第6个
第7个
第8个
第9个
第10个
第11个
第12个
第13个
第14个
第15个
第16个
第17个
第18个
第19个
第20个
第21个
第22个
第23个
第24个
第25个
第26个
第27个
第28个
第29个
第30个
第31个
第32个
第33个
第34个
第35个
第36个
第37个
第38个
第39个
第40个
第41个
第42个
第43个
第44个
第45个
第46个
第47个
第48个
第49个
第50个
第51个
第52个
第53个
第54个
第55个
第56个
第57个
第58个
第59个
第60个
第61个
第62个
第63个
第64个
第65个
第66个
第67个
第68个
第69个
第70个
第71个
第72个
第73个
第74个
第75个
第76个
第77个
第78个
第79个
第80个
第81个
第82个
第83个
第84个
第85个
第86个
第87个
第88个
第89个
第90个
第91个
第92个
第93个
第94个
第95个
第96个
第97个
第98个
第99个
第100个
第101个
第102个
第103个
第104个
第105个
第106个
第107个
第108个
第109个
第110个
第111个
第112个
第113个
第114个
第115个
第116个
第117个
第118个
第119个
第120个
第121个
第122个
第123个
第124个
第125个
第126个
第127个
第128个
第129个
第130个
第131个
第132个
第133个
第134个
第135个
第136个
第137个
第138个
第139个
第140个
第141个
第142个
第143个
第144个
第145个
第146个
第147个
第148个
第149个
第150个


In [58]:
lst_key=[]
lst_value=[]
for key,value in sss.items():
    lst_key.append(key)
    lst_value.append(value)

In [17]:
sss

{'建议将春节假期延长至9天': {'href': 'https://s.weibo.com/weibo?q=%23%E5%BB%BA%E8%AE%AE%E5%B0%86%E6%98%A5%E8%8A%82%E5%81%87%E6%9C%9F%E5%BB%B6%E9%95%BF%E8%87%B39%E5%A4%A9%23&t=31&band_rank=1&Refer=top',
  'hot': 2293864},
 '阳过一次管多久': {'href': 'https://s.weibo.com/weibo?q=%23%E9%98%B3%E8%BF%87%E4%B8%80%E6%AC%A1%E7%AE%A1%E5%A4%9A%E4%B9%85%23&t=31&band_rank=2&Refer=top',
  'hot': 1226265},
 '如何科学稳妥地恢复运动': {'href': 'https://s.weibo.com/weibo?q=%23%E5%A6%82%E4%BD%95%E7%A7%91%E5%AD%A6%E7%A8%B3%E5%A6%A5%E5%9C%B0%E6%81%A2%E5%A4%8D%E8%BF%90%E5%8A%A8%23&t=31&band_rank=3&Refer=top',
  'hot': 1115358},
 '90后也开始报复性存钱': {'href': 'https://s.weibo.com/weibo?q=%2390%E5%90%8E%E4%B9%9F%E5%BC%80%E5%A7%8B%E6%8A%A5%E5%A4%8D%E6%80%A7%E5%AD%98%E9%92%B1%23&t=31&band_rank=4&Refer=top',
  'hot': 1010749},
 '前夫接孩子带走2儿子独留女儿': {'href': 'https://s.weibo.com/weibo?q=%23%E5%89%8D%E5%A4%AB%E6%8E%A5%E5%AD%A9%E5%AD%90%E5%B8%A6%E8%B5%B02%E5%84%BF%E5%AD%90%E7%8B%AC%E7%95%99%E5%A5%B3%E5%84%BF%23&t=31&band_rank=5&Refer=top',
  'hot': 97

In [11]:
for i in range(len(lst_value)):
    print(i)
    print(lst_value[i]['zhong'])

0

1

2

3
电影
4

5

6

7

8

9

10
剧集
11
剧集
12

13

14

15

16

17

18

19

20
晚会
21
音乐
22

23
电影
24

25

26

27

28

29

30

31
剧集
32
音乐
33

34

35
电影
36

37

38

39

40

41

42

43

44

45

46

47

48

49



In [14]:
with open('raw/20230113.json', encoding='utf-8') as f:
    json_data = json.load(f)
sss=json_data
lst_key=[]
lst_value=[]
create_date = "2023-01-13"
for key,value in sss.items():
    lst_key.append(key)
    lst_value.append(value)
for kk in range(len(lst_key)):
    
    sql = "INSERT INTO weibohotpot(title,url,hot,create_date,zhong) \
                      VALUES ('%s', '%s', %d,'%s','%s')" % (lst_key[kk], lst_value[kk]['href'], lst_value[kk]['hot'], create_date,lst_value[kk]['zhong'])
    db = MysqlUtil()
    db.insert(sql)

In [ ]:
def add_article():
    form = ArticleForm(request.form)
    if request.method == "POST" and form.validate():
        # 获取表单字段内容
        title = form.title.data
        content = form.content.data
        author = session['username']
        create_date = time.strftime("%Y-%m-%d", time.localtime())
        db = MysqlUtil()
        # 插入数据的sql语句
        sql = "INSERT INTO articles(title,url,hot,create_date) \
                      VALUES ('%s', '%s', '%s','%s')" % (title, content, author, create_date)  # 插入数据的SQL语句
        db.insert(sql)

In [105]:
title = "asd"
content = "asd"
author = int(123)
create_date = time.strftime("%Y-%m-%d", time.localtime())

In [109]:
sql = "INSERT INTO weibohotpot(title,url,hot,create_date) \
                      VALUES ('%s', '%s', %d,'%s')" % (title, content, author, create_date)  # 插入数据的SQL语句

In [87]:
sql = "INSERT INTO weibohotpot(title,url,hot,create_date) \
                      VALUES ('aasd', 'asdasd', 123,'2023-01-11')"

In [88]:
sql

"INSERT INTO weibohotpot(title,url,hot,create_date)                       VALUES ('aasd', 'asdasd', 123,'2023-01-11')"

In [91]:
time.strftime("%Y-%m-%d", time.localtime())

'2023-01-12'

In [130]:
try:
    db=pymysql.connect(host='192.168.8.161',user='root',password='Wzh010310*',
                       db='weibo',port=3306)
except Exception as e:
    print('数据库连接失败',e)
db=connet_mysql()
cursor = db.cursor(cursor=pymysql.cursors.DictCursor)
sql = "INSERT INTO weibohotpot(title,url,hot,create_date) \
                      VALUES ('aasd', 'asdasd', 123,'2023-01-11')"

NameError: name 'connet_mysql' is not defined

In [126]:
s=time.strftime("%Y-%m-%d", time.localtime())

TypeError: localtime() takes no keyword arguments

In [125]:
s

str

In [30]:

m = re.findall("\d+", str(time.strftime("%Y-%m-%d", time.localtime())))

'2023-01-12'

In [42]:
starttime="20221211"
endtime="20221208"
d1=datetime.datetime.strptime(starttime,"%Y%m%d")
d2=datetime.datetime.strptime(endtime,"%Y%m%d")
diff_days=d1-d2
print(d1+"1")

TypeError: unsupported operand type(s) for +: 'datetime.datetime' and 'str'

In [2]:
#!/usr/bin/env python
#-*-coding:utf-8
# author zhaoli

import datetime
import time


def str2date(str,date_format="%Y%m%d"):
    date = datetime.datetime.strptime(str, date_format)
    return date

def date2str(date,date_formate = "%Y%m%d"):
    str = date.strftime(date_formate)
    return str



def date_delta(date,gap,formate = "%Y%m%d"):
    date = str2date(date)
    pre_date = date + datetime.timedelta(days=-gap)
    pre_str = date2str(pre_date,formate)  # date形式转化为str
    return pre_str


def str2timestamp(str,timestamp_len=10):
    date_array = time.strptime(str,"%Y-%m-%d %H:%M:%S")
    timestamp = int(time.mktime(date_array))
    if timestamp_len==13:
        timestamp *=1000
    return timestamp


In [62]:
m=re.findall("\d+", str(time.strftime("%Y-%m-%d", time.localtime())))
m=m[0]+m[1]+m[2]
print(type(m))

<class 'str'>


In [71]:
m=re.findall("\d+", str(time.strftime("%Y-%m-%d", time.localtime())))
m=m[0]+m[1]+m[2]
def date_delta(date,gap,formate = "%Y%m%d"):
    date = str2date(date)
    pre_date = date + datetime.timedelta(days=-gap)
    pre_str = date2str(pre_date,formate)  # date形式转化为str
    return pre_str

m=date_delta(m,2)

In [75]:
kk=m[0:4]+"-"+m[4:6]+"-"+m[6:8]

In [76]:
kk

'2023-01-10'

In [1]:
range(10)

range(0, 10)

In [92]:
m=re.findall("\d+", str(time.strftime("%Y-%m-%d", time.localtime())))
m=m[0]+m[1]+m[2]

In [95]:
'raw/'+m+'.json'

'raw/20230112.json'

In [52]:
sql_2 = f'select create_date,count(create_date) as t from weibohotpot group by create_date'
db = MysqlUtil()
count_number = db.fetchall(sql_2)
numbers=count_number

In [58]:
def date_delta(date,gap,formate = "%Y%m%d"):
    date = str2date(date)
    pre_date = date + datetime.timedelta(days=-gap)
    pre_str = date2str(pre_date,formate)  # date形式转化为str
    return pre_str

def str2date(str,date_format="%Y%m%d"):
    date = datetime.datetime.strptime(str, date_format)
    return date

def date2str(date,date_formate = "%Y%m%d"):
    str = date.strftime(date_formate)
    return str
date_list=[]
number_list=[]
for i in range(len(count_number)):
    date_list.append(date2str(count_number[i]['create_date']))
    number_list.append(count_number[i]['t'])

20220211
212
20220212
654
20220213
600
20220214
640
20220215
690
20220216
694
20220217
714
20220218
666
20220219
571
20220220
334
20220221
303
20220222
300
20220223
301
20220224
341
20220225
306
20220226
306
20220227
302
20220228
352
20220301
322
20220302
299
20220303
294
20220304
325
20220305
288
20220306
303
20220307
318
20220308
315
20220309
292
20220310
274
20220311
336
20220312
310
20220313
301
20220314
311
20220315
318
20220316
300
20220317
301
20220318
330
20220319
305
20220320
299
20220321
322
20220322
299
20220323
294
20220324
307
20220325
305
20220326
273
20220327
238
20220328
301
20220329
327
20220330
321
20220331
339
20220401
297
20220402
333
20220403
313
20220404
296
20220405
297
20220406
330
20220407
335
20220408
319
20220409
309
20220410
306
20220411
324
20220412
318
20220413
343
20220414
339
20220415
325
20220416
334
20220417
315
20220418
307
20220419
297
20220420
322
20220421
324
20220422
339
20220423
316
20220424
334
20220425
320
20220426
326
20220427
313
20220428
331

In [59]:
number_list

[212,
 654,
 600,
 640,
 690,
 694,
 714,
 666,
 571,
 334,
 303,
 300,
 301,
 341,
 306,
 306,
 302,
 352,
 322,
 299,
 294,
 325,
 288,
 303,
 318,
 315,
 292,
 274,
 336,
 310,
 301,
 311,
 318,
 300,
 301,
 330,
 305,
 299,
 322,
 299,
 294,
 307,
 305,
 273,
 238,
 301,
 327,
 321,
 339,
 297,
 333,
 313,
 296,
 297,
 330,
 335,
 319,
 309,
 306,
 324,
 318,
 343,
 339,
 325,
 334,
 315,
 307,
 297,
 322,
 324,
 339,
 316,
 334,
 320,
 326,
 313,
 331,
 324,
 310,
 296,
 281,
 321,
 298,
 314,
 311,
 313,
 306,
 309,
 316,
 316,
 299,
 308,
 288,
 272,
 272,
 289,
 308,
 324,
 328,
 313,
 270,
 295,
 294,
 267,
 297,
 262,
 261,
 298,
 287,
 294,
 301,
 254,
 279,
 241,
 308,
 320,
 290,
 271,
 306,
 328,
 253,
 278,
 282,
 301,
 281,
 316,
 320,
 289,
 266,
 290,
 322,
 306,
 315,
 334,
 276,
 317,
 293,
 287,
 316,
 316,
 307,
 299,
 320,
 296,
 317,
 295,
 298,
 285,
 323,
 282,
 310,
 298,
 319,
 282,
 273,
 280,
 242,
 293,
 312,
 295,
 316,
 281,
 318,
 288,
 314,
 319,
 316

In [8]:
import time
time.strftime("%Y-%m-%d", time.localtime())
time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))

'2023-01-23 11:10:39'

In [3]:
sql = f'SELECT * FROM weibohotpot  where create_date="20230113"'

In [4]:
db = MysqlUtil()
countd_number = db.fetchall(sql)
numbers2=countd_number

In [5]:
numbers2

[{'id': 6901,
  'title': 'TheShy多个韩服号被永封',
  'url': 'https://s.weibo.com/weibo?q=%23TheShy%E5%A4%9A%E4%B8%AA%E9%9F%A9%E6%9C%8D%E5%8F%B7%E8%A2%AB%E6%B0%B8%E5%B0%81%23&t=31&band_rank=44&Refer=top',
  'hot': 129296,
  'create_date': datetime.date(2023, 1, 13),
  'zhong': ''},
 {'id': 6902,
  'title': '1888万彩礼帖文作者算造谣吗',
  'url': 'https://s.weibo.com/weibo?q=%231888%E4%B8%87%E5%BD%A9%E7%A4%BC%E5%B8%96%E6%96%87%E4%BD%9C%E8%80%85%E7%AE%97%E9%80%A0%E8%B0%A3%E5%90%97%23&t=31&band_rank=45&Refer=top',
  'hot': 122572,
  'create_date': datetime.date(2023, 1, 13),
  'zhong': ''},
 {'id': 6903,
  'title': '宁德时代单季盈利破百亿',
  'url': 'https://s.weibo.com/weibo?q=%23%E5%AE%81%E5%BE%B7%E6%97%B6%E4%BB%A3%E5%8D%95%E5%AD%A3%E7%9B%88%E5%88%A9%E7%A0%B4%E7%99%BE%E4%BA%BF%23&t=31&band_rank=46&Refer=top',
  'hot': 113552,
  'create_date': datetime.date(2023, 1, 13),
  'zhong': ''},
 {'id': 6904,
  'title': '去世母亲的每件财物都标注儿女名字',
  'url': 'https://s.weibo.com/weibo?q=%23%E5%8E%BB%E4%B8%96%E6%AF%8D%E4%BA%B2%E7%9A%84%E6

list

In [66]:
import time, json
import pandas as pd
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.commons.utils import JsCode
def render_lines():
    date_list=[]
    number_list=[]
    for i in range(len(count_number)):
        date_list.append(date2str(count_number[i]['create_date']))
        number_list.append(count_number[i]['t'])   
    line = (
            Line()
        .add_xaxis(date_list)
        .add_yaxis('热搜数量', number_list, is_smooth=True,
                markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max"),
                                                        opts.MarkPointItem(type_="min")]))

        # 隐藏数字 设置面积
        .set_series_opts(
            areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False))
        # 设置x轴标签旋转角度
        .set_global_opts(xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-30)), 
                        yaxis_opts=opts.AxisOpts(name='数量', min_=0), 
                        title_opts=opts.TitleOpts(title='每日微博热搜图'))          
        )


    return line

In [5]:
import re 
import time
m=re.findall("\d+", str(time.strftime("%Y-%m-%d", time.localtime())))
m=m[0]+m[1]+m[2]
kk=m[0:4]+"-"+m[4:6]+"-"+m[6:8]
sql_3 = f'select zhong,count(zhong) as t from weibohotpot  WHERE create_date="{kk}" group by zhong'
db = MysqlUtil()
count_number = db.fetchall(sql_3)

In [8]:
count_number

[{'zhong': None, 't': 0},
 {'zhong': '', 't': 85},
 {'zhong': '剧集', 't': 7},
 {'zhong': '晚会', 't': 1},
 {'zhong': '电影', 't': 4},
 {'zhong': '盛典', 't': 1},
 {'zhong': '综艺', 't': 5},
 {'zhong': '音乐', 't': 5}]

In [40]:
number_list=[]
data_list=[]

In [41]:
for i in range(len(count_number)):
    if count_number[i]['zhong'] != None and count_number[i]['zhong'] != "":
        number_list.append(count_number[i]['zhong'])
        data_list.append(count_number[i]['t'])
for i in range(len(number_list)):
    a = {"name": number_list[i], "max": 10, "min": 0}
    c_schema.append(a)

In [42]:
data_list

[7, 1, 4, 1, 5, 5]

In [45]:
data = [{"value": data_list, "name": "微博热搜种类画像"}]

In [31]:
c_schema = [
        {"name": "音乐", "max": 10, "min": 0},
        {"name": "综艺", "max": 10, "min": 0},
        {"name": "盛典", "max": 10, "min": 0},
        {"name": "电影", "max": 10, "min": 0},
        {"name": "晚会", "max": 10, "min": 0},
        {"name": "剧集", "max": 10, "min": 0},
    ]

In [72]:

# entity_dict_1 = sorted(kk.items(), key=lambda x: x[1])

In [73]:
entity_dict_1

[('晚会', 1), ('盛典', 1), ('电影', 4), ('综艺', 5), ('音乐', 5), ('剧集', 7)]

In [79]:
kk

{'剧集': 7, '晚会': 1, '电影': 4, '盛典': 1, '综艺': 5, '音乐': 5}

In [89]:
sss=zip(number_list,data_list)
kk=dict(sss)
entity_dict_1 = sorted(kk.items(), key=lambda x: x[1])
print(entity_dict_1[-1][0])

 


剧集


In [6]:
sql_8=f'SELECT count(`id`) as t FROM `weiboevents` WHERE gender="f"'
db = MysqlUtil()
count_famale = db.fetchall(sql_8)


In [3]:
sql_q = f'select `微博情绪`,count(`微博情绪`) as t from bigevent WHERE title="武汉蝉联全国文明城市" group by `微博情绪`'
db = MysqlUtil()
qgfb = db.fetchall(sql_q)

sql_cs=f'select `微博情绪`,count(`微博情绪`) as t from bigevent WHERE title="武汉蝉联全国文明城市" AND `城市`="郑州" group by `微博情绪`'
db = MysqlUtil()
qgfb_cs = db.fetchall(sql_cs)

[{'微博情绪': '中性', 't': 1649},
 {'微博情绪': '喜悦', 't': 391},
 {'微博情绪': '恐惧', 't': 2},
 {'微博情绪': '悲伤', 't': 15},
 {'微博情绪': '惊奇', 't': 21},
 {'微博情绪': '愤怒', 't': 348}]

In [5]:
qgfb_cs

[{'微博情绪': '中性', 't': 20}]

In [30]:
zong_1=0
jiji_1=0
xiaoji_1=0
zong_2=0
jiji_2=0
xiaoji_2=0
for i in qgfb:
    zong_1+=i['t']
    if i['微博情绪']=='喜悦':
        jiji_1+=i['t']
    if i['微博情绪']=='恐惧':
        xiaoji_1+=i['t']
    if i['微博情绪']=='悲伤':
        xiaoji_1+=i['t']
    if i['微博情绪']=='惊奇':
        jiji_1+=i['t']
    if i['微博情绪']=='愤怒':
        xiaoji_1+=i['t']
for i in qgfb_cs:
    zong_2+=i['t']
    if i['微博情绪']=='喜悦':
        jiji_2+=i['t']
    if i['微博情绪']=='恐惧':
        xiaoji_2+=i['t']
    if i['微博情绪']=='悲伤':
        xiaoji_2+=i['t']
    if i['微博情绪']=='惊奇':
        jiji_2+=i['t']
    if i['微博情绪']=='愤怒':
        xiaoji_2+=i['t']
if (jiji_2/zong_2 < jiji_1/zong_1):
    print("该城市非常乐观")
if (jiji_2/zong_2 < jiji_1/zong_1):
    print("该城市非常乐观")
if (jiji_2/zong_2 = jiji_1/zong_1):
    print("该城市情感既不乐观也不悲观")

该城市非常累官


2426

2426